In [1]:
#!pip install --upgrade pip
!pip install gensim
!pip install nltk

In [2]:
import pprint
import nltk
from nltk.corpus import stopwords
from gensim import corpora

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/imayak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
text_corpus = [
    "Human machine interface for lab abc computer and machine applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [5]:
stop_words = stopwords.words('english')
stop_words = stop_words + ['com', 'edu', 'subject', 'lines', 'organization', 'would', 'article', 'could']

In [6]:
# Create a set of frequent words
#stop_words = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stop_words]
         for document in text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

[['human', 'machine', 'interface', 'computer', 'machine'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [7]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)
print(dictionary.token2id)

Dictionary(13 unique tokens: ['computer', 'human', 'interface', 'machine', 'response']...)
{'computer': 0, 'human': 1, 'interface': 2, 'machine': 3, 'response': 4, 'survey': 5, 'system': 6, 'time': 7, 'user': 8, 'eps': 9, 'trees': 10, 'graph': 11, 'minors': 12}


In [8]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


In [9]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1), (2, 1), (3, 2)],
 [(0, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 1), (8, 1), (9, 1)],
 [(1, 1), (6, 2), (9, 1)],
 [(4, 1), (7, 1), (8, 1)],
 [(10, 1)],
 [(10, 1), (11, 1)],
 [(10, 1), (11, 1), (12, 1)],
 [(5, 1), (11, 1), (12, 1)]]


In [10]:
from gensim import models

tfidf = models.TfidfModel(bow_corpus)

words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

[(6, 0.5898341626740045), (12, 0.8075244024440723)]


In [11]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)

In [12]:
query_document = 'system engineering'.split()
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.32448703), (2, 0.41707572), (3, 0.7184812), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


In [14]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)

3 0.7184812
2 0.41707572
1 0.32448703
0 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0


In [39]:
corpus_tfidf = tfidf[bow_corpus]
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]

In [ ]:
lsi_model.print_topics()

In [ ]:
for doc, as_text in zip(corpus_lsi, text_corpus):
    print(doc, as_text)

In [34]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi_model[bow_corpus]) 

In [59]:
doc = "My PC is getting old"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi_model[vec_bow]  
vec_lsi
for doc, as_text in zip(vec_lsi, text_corpus):
    print(doc, as_text)

In [60]:
sims = index[vec_lsi]
for doc, as_text in zip(sims, text_corpus):
    print(doc, as_text)

0.0 Human machine interface for lab abc computer and machine applications
0.0 A survey of user opinion of computer system response time
0.0 The EPS user interface management system
0.0 System and human system engineering testing of EPS
0.0 Relation of user perceived response time to error measurement
0.0 The generation of random binary unordered trees
0.0 The intersection graph of paths in trees
0.0 Graph minors IV Widths of trees and well quasi ordering
0.0 Graph minors A survey
